In [1]:
import pickle
import sys
from pathlib import Path
from typing import Any

import tqdm

from preprocessing.normalise import collect_id, normalize_id
from utils.loader import load_raw_corpus
from utils.logging import setup_logging

sys.setrecursionlimit(20000)
setup_logging()

In [2]:
TERM_TYPE = set(
    [
        "DebuggerStatement",
        "ThisExpression",
        "Super",
        "EmptyStatement",
        "Import",
    ]
)

MAX_SEQ_LEN = 512

In [5]:
corpus = load_raw_corpus(Path("../corpus/DIE"))
for ast in tqdm.tqdm(corpus):
    id_idx = {"v": 0, "f": 0, "c": 0}
    id_map = {}
    collect_id(ast, id_map, id_idx)
    normalize_id(ast, id_map)

 32%|███▏      | 4584/14199 [00:24<02:52, 55.66it/s] /home/pranav/Projects/Uni/Year_4/Computing/FinarYearProject/js-rl/src/utils/loader.py:30: FutureWarning: Possible set difference at position 2
  ast = load_ast(code, file, ast_path)
 40%|████      | 5721/14199 [00:31<00:28, 302.62it/s]/home/pranav/Projects/Uni/Year_4/Computing/FinarYearProject/js-rl/src/utils/loader.py:30: FutureWarning: Possible nested set at position 2
  ast = load_ast(code, file, ast_path)
/home/pranav/Projects/Uni/Year_4/Computing/FinarYearProject/js-rl/src/utils/loader.py:30: FutureWarning: Possible nested set at position 1
  ast = load_ast(code, file, ast_path)
100%|██████████| 14017/14017 [00:37<00:00, 372.40it/s] 


In [6]:
with open("../ASTBERTa/corpus.pkl", "wb") as f:
    pickle.dump(corpus, f)

In [3]:
with open("../ASTBERTa/corpus.pkl", "rb") as f:
    corpus = pickle.load(f)

/home/pranav/.cache/pypoetry/virtualenvs/js-rl-vfj9GiAe-py3.10/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: Possible set difference at position 2
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pranav/.cache/pypoetry/virtualenvs/js-rl-vfj9GiAe-py3.10/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: Possible nested set at position 2
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/pranav/.cache/pypoetry/virtualenvs/js-rl-vfj9GiAe-py3.10/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: Possible nested set at position 1
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
from js_ast.fragmentise import node_to_frags


frag_seqs: list[list[dict[str, Any]]] = []
frag_info_seqs: list[list[tuple[int, str]]] = []
all_node_types: set[str] = set()

for ast in tqdm.tqdm(corpus):
    frag_seq: list[dict[str, Any]] = []
    node_types: set[str] = set()

    node_to_frags(ast, frag_seq, node_types)

    frag_seqs.append(frag_seq)
    all_node_types.update(node_types)

100%|██████████| 14017/14017 [00:27<00:00, 501.57it/s] 


In [16]:
frag_seqs_len = list(sorted(map(lambda x: len(x), frag_seqs)))
frag_seqs_below_max = [frag_seqs for x in frag_seqs if len(x) < 512]

print("Length of frag_seqs:", len(frag_seqs))
print("Node types:", all_node_types)

print("Max length of frag_seqs:", frag_seqs_len[-100:])
print("Min length of frag_seqs:", frag_seqs_len[:5])
print("Avg length of frag_seqs:", sum(frag_seqs_len) / len(frag_seqs_len))
print("Percentage of frag_seqs below 1024:", len(frag_seqs_below_max) / len(frag_seqs))

Length of frag_seqs: 14017
Node types: {'ClassBody', 'AssignmentPattern', 'YieldExpression', 'ThrowStatement', 'ClassExpression', 'ObjectPattern', 'MemberExpression', 'WhileStatement', 'AwaitExpression', 'Literal', 'TryStatement', 'MetaProperty', 'ArrayPattern', 'BlockStatement', 'Property', 'CallExpression', 'ForStatement', 'Program', 'SequenceExpression', 'NewExpression', 'DoWhileStatement', 'ArrayExpression', 'LogicalExpression', 'ObjectExpression', 'FunctionExpression', 'TemplateLiteral', 'ClassDeclaration', 'MethodDefinition', 'SwitchStatement', 'ContinueStatement', 'ReturnStatement', 'SwitchCase', 'BreakStatement', 'ForOfStatement', 'ConditionalExpression', 'TaggedTemplateExpression', 'VariableDeclarator', 'IfStatement', 'ArrowFunctionExpression', 'ExpressionStatement', 'VariableDeclaration', 'TemplateElement', 'BinaryExpression', 'RestElement', 'UnaryExpression', 'AssignmentExpression', 'LabeledStatement', 'UpdateExpression', 'CatchClause', 'Identifier', 'WithStatement', 'Spread

In [17]:
from collections import defaultdict

from js_ast.fragmentise import hash_frag

frag_freq: dict[str, int] = defaultdict(int)
hash_to_frag: dict[str, dict[str, Any]] = {}
frag_hash_to_type: dict[str, str] = {}

for frag_seq in tqdm.tqdm(frag_seqs):
    for frag in frag_seq:
        frag_hash = hash_frag(frag)
        frag_freq[frag_hash] += 1

        if frag_hash not in hash_to_frag:
            hash_to_frag[frag_hash] = frag

        if frag_hash not in frag_hash_to_type:
            frag_hash_to_type[frag_hash] = frag["type"]

100%|██████████| 14017/14017 [00:24<00:00, 575.14it/s] 


In [18]:
frag_freq_list = list(sorted(frag_freq.items(), reverse=True, key=lambda x: x[1]))
unique_vocab_frags = set([frag_hash for frag_hash, freq in frag_freq_list if freq > 3])
oov_frags: list[str] = []

# Add OOV anonymous frag type for those not in vocabulary
for frag_type in all_node_types:
    oov_frag = {"type": frag_type}
    oov_frag_hash = hash_frag(oov_frag)
    oov_frags.append(oov_frag_hash)
    frag_hash_to_type[oov_frag_hash] = frag_type
    hash_to_frag[oov_frag_hash] = oov_frag

unique_vocab_frags.update(oov_frags)
vocab_frags = list(unique_vocab_frags)

print("Number of unique fragments:", len(frag_freq))
print("Max frequency:", max(frag_freq.values()))
print("Min frequency:", min(frag_freq.values()))

print("Number of unique fragments with freq > 5:", len(vocab_frags))
print([hash_to_frag[frag] for frag in vocab_frags[:10]])

Number of unique fragments: 376817
Max frequency: 446351
Min frequency: 1
Number of unique fragments with freq > 5: 29289
[{'type': 'Literal', 'value': 'ok', 'raw': "'ok'", 'regex': None, 'bigint': None}, {'type': 'Literal', 'value': '__hgzm=144631658.1231367708.1.1.hgzpfe=(qverpg)|hgzppa=(qverpg)|hgzpzq=(abar)', 'raw': "'__hgzm=144631658.1231367708.1.1.hgzpfe=(qverpg)|hgzppa=(qverpg)|hgzpzq=(abar)'", 'regex': None, 'bigint': None}, {'type': 'Literal', 'value': '\udfff', 'raw': '"\\udfff"', 'regex': None, 'bigint': None}, {'type': 'Literal', 'value': 'getOwnPropertyDescriptor', 'raw': '"getOwnPropertyDescriptor"', 'regex': None, 'bigint': None}, {'type': 'Literal', 'value': re.compile('Ԩ', re.IGNORECASE), 'raw': '/\\u0528/i', 'regex': {'pattern': '\\u0528', 'flags': 'i'}, 'bigint': None}, {'type': 'Literal', 'value': 914, 'raw': '914', 'regex': None, 'bigint': None}, {'type': 'LabeledStatement', 'label': {'type': 'Identifier'}, 'body': {'type': 'DoWhileStatement'}}, {'type': 'Identifie

In [19]:
PAD_TOKEN = "<pad>"
CLS_TOKEN = "<s>"
SEP_TOKEN = "</s>"
MASK_TOKEN = "<mask>"
UNK_TOKEN = "<unk>"

special_tokens = [PAD_TOKEN, CLS_TOKEN, MASK_TOKEN, SEP_TOKEN, UNK_TOKEN]

In [20]:
import numpy as np

ordered_vocab = special_tokens + list(vocab_frags)
vocab = set(ordered_vocab)

token_to_id = {token: i for i, token in enumerate(ordered_vocab)}
id_to_token = {i: token for token, i in token_to_id.items()}

special_token_ids = set([token_to_id[token] for token in special_tokens])

# Dictionary for fragment to type, special tokens are mapped to their string representation
frag_id_to_type = {token_to_id[frag]: frag_hash_to_type[frag] for frag in vocab_frags}
frag_id_to_frag = {token_to_id[frag]: hash_to_frag[frag] for frag in vocab_frags}
frag_type_to_id = {}

for frag_id, frag_type in frag_id_to_type.items():
    if frag_type not in frag_type_to_id:
        frag_type_to_id[frag_type] = set()

    frag_type_to_id[frag_type].add(frag_id)

In [21]:
frag_data = {
    "frag_seqs": frag_seqs,
    "frag_id_to_type": frag_id_to_type,
    "frag_id_to_frag": frag_id_to_frag,
    "frag_type_to_id": frag_type_to_id,
}

vocab_data = {
    "vocab": vocab,
    "token_to_id": token_to_id,
    "id_to_token": id_to_token,
    "special_token_ids": special_token_ids,
}

pickle.dump(frag_data, open("../ASTBERTa/frag_data.pkl", "wb"))
pickle.dump(vocab_data, open("../ASTBERTa/vocab_data.pkl", "wb"))

In [4]:
import pickle
import tqdm

with open("../ASTBERTa/frag_data.pkl", "rb") as f:
    frag_data = pickle.load(f)

with open("../ASTBERTa/vocab_data.pkl", "rb") as f:
    vocab_data = pickle.load(f)


frag_seqs = frag_data["frag_seqs"]
frag_id_to_type = frag_data["frag_id_to_type"]
frag_id_to_frag = frag_data["frag_id_to_frag"]
frag_type_to_id = frag_data["frag_type_to_id"]

vocab = vocab_data["vocab"]
token_to_id = vocab_data["token_to_id"]
id_to_token = vocab_data["id_to_token"]
special_token_ids = vocab_data["special_token_ids"]

In [8]:
from js_ast.fragmentise import hash_frag

data: list[list[int]] = []

for frag_seq in tqdm.tqdm(frag_seqs):
    seq: list[int] = []

    for frag in frag_seq:
        frag_hash = hash_frag(frag)
        if frag_hash in vocab:
            seq.append(token_to_id[frag_hash])
        else:
            oov_frag = {"type": frag_hash_to_type[frag_hash]}
            oov_frag_hash = hash_frag(oov_frag)
            if oov_frag_hash in vocab:
                seq.append(token_to_id[oov_frag_hash])
            else:
                print("UNK_TOKEN")
                seq.append(token_to_id[UNK_TOKEN])

    data.append([token_to_id[CLS_TOKEN]] + seq + [token_to_id[SEP_TOKEN]])

  0%|          | 0/14017 [00:00<?, ?it/s]


NameError: name 'frag_hash_to_type' is not defined

In [40]:
import pickle

pickle.dump(data, open("../ASTBERTa/data.pkl", "wb"))
# pickle.dump(token_to_id, open("ASTBERTa/token_to_id.pkl", "wb"))
# pickle.dump(vocab, open("ASTBERTa/vocab.pkl", "wb"))
# pickle.dump(hash_to_frag, open("ASTBERTa/hash_to_frag.pkl", "wb"))
# pickle.dump(id_to_token, open("ASTBERTa/id_to_token.pkl", "wb"))